# Data Structures

Common data structures like `linear containers (like array and vector)`, `stack`, `queue`, `deuque (double ended queue)`, `set/map (balanced binary search tree)`, `unordered map/set (hash tables)`, `priority queue` are already implemented in STL library of C++. Hence, implementation of those data structures are ignored since they can be used directly from the standard library. The corresponding C++ reference websites of those data structures are given in the table below:

| Data Structure              | STL Implementation |
| :-------------------------: | :----------------: |
| Fixed-size array            | [std::array](https://en.cppreference.com/w/cpp/container/array)   |
| Dynamic array               | [std::vector](https://en.cppreference.com/w/cpp/container/vector) |
| Stack                       | [std::stack](https://en.cppreference.com/w/cpp/container/stack)   |
| Qeueu                       | [std::queue](https://en.cppreference.com/w/cpp/container/queue)   |
| Deque                       | [std::deque](https://en.cppreference.com/w/cpp/container/deque)   |
| Balanced Binary Search Tree | [std::set](https://en.cppreference.com/w/cpp/container/set), [std::multiset](https://en.cppreference.com/w/cpp/container/multiset), [std::map](https://en.cppreference.com/w/cpp/container/map), [std::multimap](https://en.cppreference.com/w/cpp/container/multimap) |
| Hash Table                  | [std::unordered_set](https://en.cppreference.com/w/cpp/container/unordered_set), [std::unordered_multiset](https://en.cppreference.com/w/cpp/container/unordered_multiset) , [std::unordered_map](https://en.cppreference.com/w/cpp/container/unordered_map), [std::unordered_multimap](https://en.cppreference.com/w/cpp/container/unordered_multimap)  |
| Priority Queue              | [std::priority_queue](https://en.cppreference.com/w/cpp/container/priority_queue) |

* In addition to the data structures above, you can pack together the different type of objects in [`std::tuple`](https://en.cppreference.com/w/cpp/utility/tuple).

## Fenwick (Binary Indexed) Tree

For simplicity, we start with a generalization. Let's say $f$ is defined as $f(x, y)=x+y$ over integers.
Suppose we are given an array of integers $A[0...N-1]$. (Note that we are using zero-based indexing.) A Fenwick tree is just an array, $T[0...N-1]$, where each element is equal to the sum of the elements of $A$ in some range $[g(i), i]$:

$$ T_i = \Sigma_{j=g(i)}^{i} A_j $$

where $g$ is some function that satisfies $0\le g(i) \le i$.
Here, notice that if we choose $g(x)=x$, as an identity function, then we will get exact same copy of the array $A$. However, we are looking for a more optimized way of doing range queries, for this case, summation.

**Note:** _The Fenwick tree presented here uses zero-based indexing. Many people use a version of the Fenwick tree that uses one-based indexing. As such, you will also find an alternative implementation which uses one-based indexing in the implementation section. Both versions are equivalent in terms of time and memory complexity; however, for less confusion, I prefer the zero-based indexing approach._

Now, we can write some pseudo code for two main operations: sum of the elements of $A$ in the range $[0,r]$ and dupdate some element $A_i$:

```c
int sum(int r):
    res = 0;
    while (r >= 0)
    {
        res += T[r];
        r = g(r) - 1;
    }
    return res;

void update(int i, int delta):
    for all j for which g(j) <= i <= j
        T[j] += delta
```

The function `sum` works as follows:

1. First, it adds the sum of the range $[g(r), r]$ (i.e. $T[r]$) to the `result`.
2. Then, it jumps to the range $[g(g(r) - 1), g(r) - 1]$ and adds this range's sum to the `result`.
3. This continues until it jumps from $[0,g(g(...g(r)-1 ...-1)-1]$ to $[g(-1), -1]$; this is where it stops.

The function `update` works with the same analogy, but it "jumps" in the direction of increasing indices.

The complexity of both `sum` and `update` depends on how we choose the function $g(i)$.  The clever part of the algorithm for Fenwick trees is how it uses a special definition of the function  $g$  which can handle both operations in  $O(\log N)$  time.

---

### Choosing the correct $g$ function

The computation of  $g(i)$  is defined using the following simple operation: we replace all trailing  $1$  bits in the binary representation of  $i$  with  $0$  bits.

In other words, if the least significant digit of  $i$  in binary is  $0$ , then  $g(i) = i$ . And otherwise the least significant digit is a  $1$ , and we take this  $1$  and all other trailing  $1$ s and flip them.

For instance we get
$$\begin{align} g(11) = g(1011_2) = 1000_2 &= 8 \\\\ g(12) = g(1100_2) = 1100_2 &= 12 \\\\ g(13) = g(1101_2) = 1100_2 &= 12 \\\\ g(14) = g(1110_2) = 1110_2 &= 14 \\\\ g(15) = g(1111_2) = 0000_2 &= 0 \\\\ \end{align}$$ 
There exists a simple implementation using bitwise operations for the non-trivial operation described above:
$$g(i) = i ~\&~ (i+1),$$ 

Now, we just need to find a way to iterate over all  $j$ 's, such that  $g(j) \le i \le j$ .

It is easy to see that we can find all such  $j$ 's by starting with  $i$  and flipping the last unset bit. We will call this operation  $h(j)$ . For example, for  $i = 10$  we have:

$$\begin{align} 10 &= 0001010_2 \\\\ h(10) = 11 &= 0001011_2 \\\\ h(11) = 15 &= 0001111_2 \\\\ h(15) = 31 &= 0011111_2 \\\\ h(31) = 63 &= 0111111_2 \\\\ \vdots & \end{align}$$ 
Unsurprisingly, there also exists a simple way to perform  $h$  using bitwise operations:

$$h(j) = j ~\|~ (j+1),$$ 
where  $\|$  is the bitwise OR operator.

The following image shows a possible interpretation of the Fenwick tree as tree. The nodes of the tree show the ranges they cover.
![Fenwick Update Tree](./images/BinaryIndexedTreeVisualization.png)

In [1]:
#include <bits/stdc++.h>

struct FenwickTree {
    std::vector<int> BIT;

    FenwickTree(int n): BIT(n, 0) {}
    FenwickTree(const std::vector<int>& array): FenwickTree(array.size())
    {
        for (size_t i=0; i<array.size(); i++)
            update(i, array[i]);
    }

    int sum(int index)
    {
        int result = 0;
        while (index >= 0)
        {
            result += BIT[index];
            index = (index & (index + 1)) - 1;
        }
        return result;
    }

    int sum(int l_index, int r_index)
    {
        return sum(l_index) - sum(r_index - 1);
    }

    void update(size_t index, int delta)
    {
        while (index < BIT.size())
        {
            BIT[index] += delta;
            index = index | index + 1;
        }
    }
};

In [2]:
std::vector<int> arr = {2, 1, 1, 3, 2, 3, 4, 5, 6, 7, 8, 9};
FenwickTree tree(arr);
tree.BIT

{ 2, 3, 1, 7, 2, 5, 4, 21, 6, 13, 8, 30 }

In [3]:
// sum range query starting from zero to given index
tree.sum(4) // sum of first 5 elements = 9

9

In [4]:
// sum range query between two given index (inclusive)
tree.sum(10, 7) // sum of the elements at index 7 8 9 10 = 26

26

In [5]:
// sum range query after modifying one of the elements
tree.update(7, 42 - arr[7] /* delta on the index */); // changing the element at index 7 with 42
arr[7] = 42;
tree.sum(10, 7) // after the element update the sum should be 63

63

### Linear Construction of Fenwick Tree


The above implementation requires  $O(N \log N)$  time. It's possible to improve that to  $O(N)$  time.

The idea is, that the number  $a[i]$  at index  $i$  will contribute to the range stored in  $bit[i]$ , and to all ranges that the index  $i | (i + 1)$  contributes to. So by adding the numbers in order, you only have to push the current sum further to the next range, where it will then get pushed further to the next range, and so on.

In [6]:
FenwickTree constructFenwickTreeLinearTime(const std::vector<int>& array)
{
    std::vector<int> BIT(array.size(), 0);
    for (size_t i=0; i < array.size(); i++)
    {
        BIT[i] += array[i];
        int parent = i | (i+1);
        if (parent < array.size()) BIT[parent] += BIT[i];
    }

    FenwickTree tree(array.size());
    tree.BIT = std::move(BIT);
    return tree;
}

In [7]:
FenwickTree linear_tree = constructFenwickTreeLinearTime(arr);
assert(linear_tree.BIT == tree.BIT);
linear_tree.BIT

{ 2, 3, 1, 7, 2, 5, 4, 58, 6, 13, 8, 30 }

### Fenwick Tree with one-based indexing
For this approach we change the requirements and definition for  $T[]$  and  $g()$  a little bit. We want  $T[i]$  to store the sum of  $[g(i)+1; i]$ . This changes the implementation a little bit, and allows for a similar nice definition for  $g(i)$.

The computation of  $g(i)$  is defined as: toggling of the last set  $1$  bit in the binary representation of  $i$ .

$$\begin{align} g(7) = g(111_2) = 110_2 &= 6 \\\\ g(6) = g(110_2) = 100_2 &= 4 \\\\ g(4) = g(100_2) = 000_2 &= 0 \\\\ \end{align}$$ 
The last set bit can be extracted using  $i ~\&~ (-i)$ , so the operation can be expressed as:

 $$g(i) = i - (i ~\&~ (-i)).$$ 
And it's not hard to see, that you need to change all values  $T[j]$  in the sequence  $i,~ h(i),~ h(h(i)),~ \dots$  when you want to update  $A[j]$ , where  $h(i)$  is defined as:

 $$h(i) = i + (i ~\&~ (-i)).$$ 

As you can see, the main benefit of this approach is that the binary operations complement each other very nicely.
Implementation of this approach is left for the reader.

---

### Three Trees hidden in Fenwick Array

A _"Fenwick tree"_ is actually three implicit trees over the same array: the interrogation tree used for translating indexes to prefix sums, the update tree used for updating elements, and the search tree for translating prefix sums to indexes (rank queries). The first two are normally walked upwards, while the third is usually walked downwards.

__Interrogation Tree:__ The interrogation tree is defined so that the parent of node $i$ is $ i - lsb(i) $ where $lsb(i)$ is defined above. Implicit node 0 is the root. Node $i$ has $log_2(lsb(i))$ children $(i+1, i+2, i+4, ..., i+lsb(i)/2)$, and $lsb(i)$ total descendants.

![One-based Fenwick Interrogation Tree](./images/FenwickInterrogationTree.png)

__Update Tree:__ The update tree is the mirror image of the interrogation tree. The parent of node $i$ is $i + lsb(i) = (i|i-1)+1$ where $|$ denotes bitwise or operation. This conceptual tree is infinite, but only the part with indexes up to 
$n$ is stored or used. Excluding the fictitious nodes with indexes greater than $n$ it will be a forest of disjoint trees, one for each bit set in the binary representation of $n$. To modify one of the values $A[i]$ add the change to $F[i]$, then $i$'s parent, then its grandparent, and so on, until the index exceeds $n$.

_The visualization of the Update Tree is given in the image in second cell._

__Search Tree:__ Unlike the other two trees, the search tree is a binary tree, arranged in an order sideways heap. Each node is assigned a height equal to the number of trailing zeros in the binary representation of its index, with the parent and children being the numerically closest index(es) of the adjacent height. Nodes with odd indexes $(lsb(i) = 1)$ are leaves. Nodes with even indexes have the closest two nodes of the next-lowest index as children, $ i \pm lsb(i)/2 $. Node $i$'s parent in the same search tree is

$$(i - lsb(i))|(2lsb(i))$$

Although this tree is potentially infinite, we may define its root to be the highest existing node. The search tree may be considered a combination of the previous two trees. A node's left subtree contains all of its descendants in the update tree, while its right subtree contains all of its descendants in the interrogation tree. A node's parent in the search tree is either its interrogation or update parent (depending on whether the node is a right or left child, respectively), and the other type of parent may be found by multiple upward steps in the search tree.

However, upward traversals in the search tree are uncommon; its primary use is to perform rank queries: given a prefix sum, at what index does it appear? This is done by a downward traversal through the search tree. During the traversal, three variables are maintained: The current node's index, the rank being sought in the subtree rooted at the current node, and a "fallback index" to be returned if the rank sought is greater than can be found in the subtree. Each step, either the current node is a fictitious node (index greater than $n$), or we must decide if the position sought is to the left or right of the end of the current node. If the rank sought is less than the Fenwick array value $F[i]$ for the current node, we must search its left subtree. If it is greater, search its right subtree. If it is equal, the direction chosen depends on how you wish to handle searches for sums lying exactly between two nodes.

---

### Finding Sum in two-dimensional array

Fenwick Tree can be implemented for 2D arrays with an easy way. The implementation is given below.

In [8]:
struct FenwickTree2D {
    std::vector<std::vector<int>> BIT;

    FenwickTree2D(size_t size_x, size_t size_y): BIT(size_x, std::vector<int>(size_y, 0)) {}
    FenwickTree2D(const std::vector<std::vector<int>>& array): FenwickTree2D(array.size(), array[0].size())
    {
        for (size_t i = 0; i < array.size(); i++)
            for (size_t j = 0; j < array[0].size(); j++)
                update(i, j, array[i][j]);
                
    }

    int sum(size_t x, size_t y) {
        int result = 0;
        for (int i = x; i >= 0; i = (i & (i + 1)) - 1)
            for (int j = y; j >= 0; j = (j & (j + 1)) - 1)
                result += BIT[i][j];
        return result;
    }

    void update(size_t x, size_t y, int delta) {
        for (int i = x; i < BIT.size(); i = i | (i + 1))
            for (int j = y; j < BIT[0].size(); j = j | (j + 1))
                BIT[i][j] += delta;
    }
};

In [9]:
std::vector<std::vector<int>> array2d = { {1, 2, 3, 4, 5, 6}, { 12, 11, 10, 9, 8, 7 }};
FenwickTree2D tree2d(array2d);
tree2d.BIT

{ { 1, 3, 3, 10, 5, 11 }, { 13, 26, 13, 52, 13, 26 } }

In [10]:
tree2d.sum(1, 1) // sum of { {1, 2}, {12, 11}} = 26

26

In [11]:
tree2d.update(0, 0, 14); // making index [0, 0] := 15
tree2d.sum(1, 1) // sum of { {15, 2}, {12, 11}} = 40

40

#### Sources
---

- [cp-algorithms](https://cp-algorithms.com/data_structures/fenwick.html)
- [wikipedia](https://en.wikipedia.org/wiki/Fenwick_tree)